In [ ]:
import sys,array
from ROOT import TFile, TCanvas
import root_numpy as rn
import numpy as np
import math
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

matplotlib.rc('text', usetex=True)
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{xfrac,fontspec,unicode-math}"]
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{mathastext}"]

font = 22   
legendfont = 18
matplotlib.rc('font',weight='bold')
matplotlib.rc('font',size=font)
matplotlib.rcParams['text.latex.preamble'] = [r'\boldmath']

from matplotlib.ticker import NullFormatter
from scipy import stats
#from matplotlib.offsetbox import AnchoredText

In [ ]:
dataPOT = 1.616 
mcbnbcos_POT = 20.7948 #20.8003 #17.9762  #4.23214
off_to_on = 35947170./65742501 
mc_to_onbeam = dataPOT/mcbnbcos_POT

det = .2157 
corsika = 0.1
ntarg = 0.063
detsys = math.sqrt(det*det + corsika*corsika + ntarg*ntarg)

print "POTS : ", dataPOT, mcbnbcos_POT 
print "MC to OnBeam : ", mc_to_onbeam
print "Off to OnBeam: ",off_to_on
print "DetSys: ", detsys

In [ ]:
def scaleStuff(df_flux,nevents):

    var_weight = np.zeros((1000,nevents))
    print 'Entering loops...'

    for i in xrange(nevents):
        if i % 500 == 0 : 
            print 'On event %d' % i
            
        for j in xrange(1000):
            var_weight[j][i] = df_flux.flux_weights.values[i][j]
    return var_weight  

In [ ]:
# Get the flux variances
f = 'Files/MCBNBCos/BT_backgrounds_getSel2Flux.root'

t = rn.root2array(f,treename='tree')
df_flux = pd.DataFrame(t)

var_weight = scaleStuff(df_flux,df_flux.shape[0])

In [ ]:
print len(var_weight[0])

In [ ]:
# Two shower contains the additional cuts from joseph
stage= 'sel2'
#stage = 'one'

f = 'Files/MCBNBCos/BT_backgrounds_sel2_envelope.root'
on ='Files/OnBeam/BT_backgrounds_%s.root'%stage
off='Files/OffBeam/BT_backgrounds_%s.root'%stage 

name = 'sel2'

t = rn.root2array(f,treename='tree')
df = pd.DataFrame(t)

a = rn.root2array(on,treename='tree')
df0 = pd.DataFrame(a)

b = rn.root2array(off,treename='tree')
df1 = pd.DataFrame(b)

print 'On, Off, MC: ', df0.shape, df1.shape, df.shape

df['mu_endx'] = np.where(df.mu_startx-df.vtx_x == 0, df.mu_endx, df.mu_startx)
df['mu_endy'] = np.where(df.mu_starty - df.vtx_y == 0, df.mu_endy, df.mu_starty)
df['mu_endz'] = np.where(df.mu_startz - df.vtx_z == 0, df.mu_endz, df.mu_startz)

df0['mu_endx'] = np.where(df0.mu_startx - df0.vtx_x == 0, df0.mu_endx, df0.mu_startx)
df0['mu_endy'] = np.where(df0.mu_starty - df0.vtx_y == 0, df0.mu_endy, df0.mu_starty)
df0['mu_endz'] = np.where(df0.mu_startz - df0.vtx_z == 0, df0.mu_endz, df0.mu_startz)

df1['mu_endx'] = np.where(df1.mu_startx - df1.vtx_x == 0, df1.mu_endx, df1.mu_startx)
df1['mu_endy'] = np.where(df1.mu_starty - df1.vtx_y == 0, df1.mu_endy, df1.mu_starty)
df1['mu_endz'] = np.where(df1.mu_startz - df1.vtx_z == 0, df1.mu_endz, df1.mu_startz)

df['mu_startx'] = df['vtx_x']
df['mu_starty'] = df['vtx_y']
df['mu_startz'] = df['vtx_z']

df0['mu_startx'] = df0['vtx_x']
df0['mu_starty'] = df0['vtx_y']
df0['mu_startz'] = df0['vtx_z']

In [ ]:
oth_v = df.query('bkgd_id == 0') #noise
cos_v = df.query('bkgd_id == 1')
cc1pi0_v = df.query('bkgd_id == 2')
nc1pi0_v = df.query('bkgd_id == 3')
cc1pi0_outFV_v = df.query('bkgd_id == 4')
multpi0_v = df.query('bkgd_id == 5')
nue_v = df.query('bkgd_id == 6')
antinumu_v = df.query('bkgd_id == 7')
cccex_v = df.query('bkgd_id == 8 ')
nccex_v = df.query('bkgd_id == 9')
ccgamma_v = df.query('bkgd_id == 10')
kaon_v = df.query('bkgd_id == 11')
ccoth_v = df.query('bkgd_id == 12 or bkgd_id == 11')
ncoth_v = df.query('bkgd_id == 13 or bkgd_id == 9')

all_but_sig_v = df.query('bkgd_id != 2')

In [ ]:
dataPOT = 1.616 
mcbnbcos_POT = 20.7948  #4.23214
off_to_on = 35947170./65742501 

mc_to_onbeam = dataPOT/mcbnbcos_POT
print "POTS : ", dataPOT, mcbnbcos_POT 
print "MC to OnBeam : ", mc_to_onbeam
print "Off to OnBeam: ",off_to_on

var = 'mu_startx'

oth_w = [ mc_to_onbeam for i in xrange(len(oth_v[var].values)) ] 
cos_w = [ mc_to_onbeam for i in xrange(len(cos_v[var].values)) ] 
nc1pi0_w = [ mc_to_onbeam for i in xrange(len(nc1pi0_v[var].values)) ] 
cc1pi0_outFV_w = [ mc_to_onbeam for i in xrange(len(cc1pi0_outFV_v[var].values)) ] 
multpi0_w = [ mc_to_onbeam for i in xrange(len(multpi0_v[var].values)) ] 
nue_w = [ mc_to_onbeam for i in xrange(len(nue_v[var].values)) ] 
antinumu_w = [ mc_to_onbeam for i in xrange(len(antinumu_v[var].values)) ] 
cccex_w = [ mc_to_onbeam for i in xrange(len(cccex_v[var].values)) ] 
nccex_w = [ mc_to_onbeam for i in xrange(len(nccex_v[var].values)) ] 
ccgamma_w = [ mc_to_onbeam for i in xrange(len(ccgamma_v[var].values)) ] 
kaon_w = [ mc_to_onbeam for i in xrange(len(kaon_v[var].values)) ] 
ccoth_w = [ mc_to_onbeam for i in xrange(len(ccoth_v[var].values)) ] 
ncoth_w = [ mc_to_onbeam for i in xrange(len(ncoth_v[var].values)) ] 
sig_w = [ mc_to_onbeam for i in xrange(len(cc1pi0_v[var].values)) ] 

cos_ext_w = [ off_to_on for i in xrange(df1.shape[0]) ] 
all_w = [ mc_to_onbeam for i in xrange(len(df[var].values)) ] 



In [ ]:
def getVariance(var,xmin,xmax,var_weight,nbins=20,bin_width=1):
    
    bin_content = np.zeros((nbins,1000))

    for i in xrange(1000):

        counts, edges, _ = plt.hist([df_flux[var].values],bins=nbins,\
            range=(xmin,xmax),weights=[var_weight[i]],rwidth=bin_width,histtype='step')

        for bi in xrange(nbins):
            bin_content[bi][i] = counts[bi]   

    frac_unc_v = [ np.std(bin_content[bi]) / counts[bi] for bi in xrange(nbins) ]
    
    return np.array(frac_unc_v)

In [ ]:
exclude_var_v =[ 'mult','mu_startx','mu_phi','mu_starty','mu_startz','mu_endx','mu_endy','mu_endz','vtx_x',\
                'vtx_y','vtx_z','pi0_oangle','pi0_totalE','mu_phi','pi0_Easym',\
                'pi0_high_shrE_corr','pi0_cos_theta_CM','pi0_cos_theta_CM_corr',\
                'tot_hits_0','tot_hits_1','tot_hits_2','n_track_hits_0',\
                'n_track_hits_1', 'n_track_hits_2','n_shower_hits_0','n_shower_hits_1','n_shower_hits_2']  

In [ ]:
def makeEnvelopePlots(var,xmin,xmax,nbins=20,xlabel='',pottitle='',unit='cm',pot=dataPOT,\
                      ticks=[0.5,1,1.5],tickmin=0.25,tickmax=1.75,getks=False,includeDetSys=True):

    x_on, y_on = np.histogram(df0[var].values,bins=nbins,range=(xmin,xmax))
    x_off, y_off = np.histogram(df1[var].values,bins=nbins,range=(xmin,xmax))
    x_bnb, y_bnb = np.histogram(df[var].values,bins=nbins,range=(xmin,xmax))
    
    bins = (y_off[:-1] + y_off[1:])/2
    bin_width = bins[1]-bins[0]
    
    bnbcos_m = x_bnb * mc_to_onbeam 
    off_m = x_off*off_to_on
    
    # Caluclate stat uncertainty
    err_on = np.sqrt(x_on)
    err_off = np.sqrt(x_off) * off_to_on
    err_mc = np.sqrt(x_bnb) * mc_to_onbeam 
    err_mc_off = np.sqrt(err_mc * err_mc + err_off * err_off)
    err_on_off = np.sqrt(err_on * err_on + err_off * err_off)
    
    # Calculate prelim systematics
    err_mc_plus_detsys = np.sqrt( x_bnb + pow(x_bnb*detsys,2) ) * mc_to_onbeam
    err_mc_off_plus_detsys = np.sqrt(err_mc_plus_detsys * err_mc_plus_detsys + err_off * err_off)
    
    # Get GENIE Errors
    genie_up = df.genie_weight_up.values * mc_to_onbeam
    genie_dn = df.genie_weight_dn.values * mc_to_onbeam
   
    fig, ax = plt.subplots(figsize=(8,6))
    counts, edges, _ = ax.hist([df1[var].values,df[var].values],bins=nbins,label=['OffBeam','MC'],color=['grey','lightblue'],\
        ec='None',stacked=True,range=(xmin,xmax),weights=[cos_ext_w,all_w],rwidth=bin_width,histtype='stepfilled')
    genie_counts_up,_, _ = ax.hist([df1[var].values,df[var].values],bins=nbins,label=['OffBeam','MC up'],color=['grey','darkblue'],\
        stacked=True,range=(xmin,xmax),weights=[cos_ext_w,genie_up],lw=2,rwidth=bin_width,histtype='step')
    genie_counts_dn,_, _ = ax.hist([df1[var].values,df[var].values],bins=nbins,label=['OffBeam','MC dn'],color=['grey','red'],\
        stacked=True,range=(xmin,xmax),weights=[cos_ext_w,genie_dn],lw=2,rwidth=bin_width,histtype='step')

    err_genie_up = genie_counts_up[-1]- counts[-1] 
    err_genie_dn = counts[-1] - genie_counts_dn[-1] 
    
    print 'Getting the flux weights...'
    frac_unc_v = getVariance(var,xmin,xmax,var_weight,nbins,bin_width)
    err_flux = frac_unc_v * bnbcos_m
    
    err_tot_up_wo_det = np.sqrt(pow(err_mc_off,2) + pow(err_flux,2) + pow(err_genie_up,2))
    err_tot_dn_wo_det = np.sqrt(pow(err_mc_off,2) + pow(err_flux,2) + pow(err_genie_dn,2))
    err_tot_wo_det = err_tot_up_wo_det + err_tot_dn_wo_det
    
    err_tot_up = np.sqrt(pow(err_mc_off_plus_detsys,2) + pow(err_flux,2) + pow(err_genie_up,2))
    err_tot_dn = np.sqrt(pow(err_mc_off_plus_detsys,2) + pow(err_flux,2) + pow(err_genie_dn,2))
    err_tot = err_tot_up + err_tot_dn
    
    ##############################################################
    weights = [cos_ext_w,cos_w,antinumu_w,nue_w,multpi0_w,cc1pi0_outFV_w,ncoth_w,nc1pi0_w,ccoth_w,cccex_w,\
           ccgamma_w,sig_w]

    labels_tst = ['Cosmic (Data)','Cosmic ($\\nu$)','anti $\\nu_{\mu}$','$\\nu_e$',\
                  'Mult $\pi^0$','CC 1-$\pi^0$ OutFV',\
                  'NC 0$\pi^0$','NC $\pi^0$',\
                 'CC Other','CC Cex','X$\\rightarrow \geq$ 1$\gamma$','Signal']
    
    colors_tst = ['lightgrey','dimgrey','darkgoldenrod','yellow','mediumpurple','purple','lime','darkgreen','lightskyblue','blue',\
                 'darkblue','salmon'] 

    fig = plt.figure(figsize=(10,8))
       
    left, width = 0.1, 0.65
    bottom, height = 0.1, 0.65
    bottom_h = left_h = left + 0.27
    rect_main = [left, bottom_h, width, height] 
    rect_histx = [left, bottom, width, 0.25]
    
    ax = plt.axes(rect_main)
    ax2 = plt.axes(rect_histx)

    #counts, edges, _ = ax.hist([df1[var].values,cos_v[var].values,oth_v[var].values,nc1pi0_v[var].values,multpi0_v[var].values,\
    #    cccex_v[var].values,nu_EM_v[var].values,cc1pi0_v[var].values],bins=nbins,label=labels_tst,color=colors_tst,\
    #    ec='None',stacked=True,range=(xmin,xmax),weights=weights,rwidth=bin_width,histtype='stepfilled')
    counts, edges,_ = ax.hist([df1[var].values,cos_v[var].values,antinumu_v[var].values,nue_v[var].values,\
        multpi0_v[var].values,cc1pi0_outFV_v[var].values,\
        ncoth_v[var].values,nc1pi0_v[var].values,\
        ccoth_v[var].values,cccex_v[var].values,ccgamma_v[var].values,\
        cc1pi0_v[var].values],bins=nbins,label=labels_tst,color=colors_tst,ec='None',stacked=True,\
        range=(xmin,xmax),weights=weights,rwidth=bin_width,histtype='stepfilled')
    if not includeDetSys:
        ax.bar(bins - bin_width/2,err_tot_wo_det,width=bin_width,bottom = bnbcos_m + off_m - err_tot_dn_wo_det,\
        alpha=.5,  fill=False, linewidth=0,hatch='\\\\\\\\')
    else:
        ax.bar(bins - bin_width/2,err_tot,width=bin_width,bottom = bnbcos_m + off_m - err_tot_dn,\
        alpha=.5,  fill=False, linewidth=0,hatch='\\\\\\\\')
    ax.errorbar(bins,x_on,xerr=bin_width/2,yerr=err_on,fmt='ko',label='Data',markersize=6,lw=1.5)
    handles,labels = ax.get_legend_handles_labels()
    lgd = 0
    
    if unit == 'rad' or unit == 'asym' or var =='pi0_low_IP_w_vtx' or var =='pi0_high_IP_w_vtx' or\
       var == 'pi0_IP' or var == 'gamma_IP_w_vtx':
        ax.set_ylabel('Events / %0.2f %s '%(bin_width,unit),fontsize=font,weight='bold')
    elif unit == 'empty':
        ax.set_ylabel('Events',fontsize=font,weight='bold')
    elif unit.split('(')[0] == 'cos':
        ax.set_ylabel('Events / %0.2f'%bin_width,fontsize=font,weight='bold')
    else:
        ax.set_ylabel('Events / %i %s '%(bin_width,unit),fontsize=font,weight='bold')
    
    t1 = ax.text(0.645,1.02,r'%.2f \textbf{$\times 10^{20}$} POT'%pot,weight='bold',color='r',fontsize=font,transform = ax.transAxes)
    t2 = ax.text(0.001,1.01,r'MicroBooNE Preliminary',weight='bold',color='r',fontsize=font,transform = ax.transAxes)

    ax.grid(True)
    ax.set_xlim(xmin,xmax)
    nullfmt = NullFormatter() 
    ax.xaxis.set_major_formatter(nullfmt)        
    
    if var in exclude_var_v:
        lgd = ax.legend(handles=handles,fontsize=legendfont,loc='upper left',bbox_to_anchor=(1,1),title=pottitle)
    else:
        lgd = ax.legend(handles=handles,fontsize=legendfont,loc=0,title=pottitle) #,loc='upper left',bbox_to_anchor=(1,0.9))
          
    fig.subplots_adjust(top=.95)

    ratio = [ x_on[i] / counts[-1][i]  if x_on[i] != 0 else -10 for i in xrange(len(counts[-1]))]
    #ratio_unc = np.sqrt( pow(err_on/counts[-1],2) + pow(ratio/counts[-1]* err_mc_off,2) )
    ratio_unc = np.sqrt( pow(err_on/counts[-1],2) )
    ratio_unc_up =  err_tot_up_wo_det / counts[-1]
    ratio_unc_dn =  err_tot_dn_wo_det / counts[-1] 
    
    if includeDetSys:
        #ratio_unc_up = np.sqrt( pow(ratio/counts[-1]* err_tot_up,2))
        #ratio_unc_dn = np.sqrt( pow(ratio/counts[-1]* err_tot_dn,2) )
        ratio_unc_up = err_tot_up / counts[-1]
        ratio_unc_dn = err_tot_dn / counts[-1] 
        
    left, width = 0.1, 0.65
    bottom_h = left_h = left + width + 0.02
    rect_histx = [left, bottom_h, width, 0.2]

    if getks:
        ks_stat, ks_pvalue = stats.ks_2samp(counts[-1],x_on)
        chi2_stat, chi2_pvalue = stats.chisquare(x_on,counts[-1])
        plt.text(0.7,0.15,'KS pvalue: %.2f'%ks_pvalue,weight='bold',fontsize=16,transform = ax.transAxes)

    ax2.plot(bins,ratio,'o')
    ax2.axhline(1,lw=2,ls='--',color='r')
    ax2.errorbar(bins,ratio,xerr=bin_width/2,yerr=ratio_unc,fmt='ko',label='Sys Error Ratio',markersize=6,lw=1.5)
    #ax2.errorbar(bins,1,xerr=bin_width/2,yerr=[],fmt='bo',label='Ratio',markersize=6,lw=1.5)
    ax2.bar(bins - bin_width/2,ratio_unc_up+ratio_unc_dn,width=bin_width,bottom = 1-ratio_unc_dn,\
        alpha=.5,  fill=False, linewidth=0,hatch='\\\\\\\\')

    ax2.set_ylabel('Ratio Data:MC ')
    ax2.set_xlabel(xlabel,fontsize=font,weight='bold')
    ax2.set_yticks(ticks)
    
    ax2.grid(True)
    ax2.set_xlim(xmin,xmax)
    ax2.set_ylim(tickmin,tickmax)

    ax.set_ylim(0)
    if includeDetSys:
        plt.savefig('Plots/Physics_sys_w_detsys_%s_onoffseparate_%s.png'%(stage,var),bbox_extra_artists=(lgd,t1,t2), bbox_inches='tight')
    else:
        plt.savefig('Plots/Physics_sys_%s_onoffseparate_%s.png'%(stage,var),bbox_extra_artists=(lgd,t1,t2), bbox_inches='tight')
    
    return counts, err_mc_off, x_on, err_on, edges   

In [ ]:
includeDetSys = False
_ = makeEnvelopePlots('mult',0,7,nbins=7,xlabel='Observed Vertex Multiplicity',unit='empty',includeDetSys=includeDetSys)

In [ ]:
includeDetSys = False
_ = makeEnvelopePlots('nshrs',0,7,nbins=7,xlabel='Number of Reconstructed Showers',unit='empty',includeDetSys=includeDetSys)
_ = makeEnvelopePlots('mu_phi',-3.14,3.14,nbins=11,xlabel='Candidate $\mu$ Track $\\phi$ [rad]',unit='rad',includeDetSys=includeDetSys)
_ = makeEnvelopePlots('mu_angle',-1,1,nbins=15,xlabel='Candidate $\mu$ Track cos($\\theta$)',unit='cos(stuff)',includeDetSys=includeDetSys)
_ = makeEnvelopePlots('mu_len',0,600,nbins=11,xlabel='Contained Candidate $\mu$ Track Length [cm]',includeDetSys=includeDetSys)
_ = makeEnvelopePlots('mult',0,7,nbins=7,xlabel='Observed Vertex Multiplicity',unit='empty',includeDetSys=includeDetSys)
_ = makeEnvelopePlots('mu_startx',0,260,nbins=11,xlabel='Candidate $\mu$ Track Start X [cm]',includeDetSys=includeDetSys)
_ = makeEnvelopePlots('mu_starty',-110,110,nbins=11,xlabel='Candidate $\mu$ Track Start Y [cm]',includeDetSys=includeDetSys)
_ = makeEnvelopePlots('mu_startz',0,1040,nbins=11,xlabel='Candidate $\mu$ Track Start Z [cm]',includeDetSys=includeDetSys)
_ = makeEnvelopePlots('mu_endx',0,260,nbins=11,xlabel='Candidate $\mu$ Track End X [cm]',includeDetSys=includeDetSys)
_ = makeEnvelopePlots('mu_endy',-120,120,nbins=11,xlabel='Candidate $\mu$ Track End Y [cm]',includeDetSys=includeDetSys)
_ = makeEnvelopePlots('mu_endz',0,1040,nbins=11,xlabel='Candidate $\mu$ Track End Z [cm]',includeDetSys=includeDetSys)